# Flank Quickstart on Google Colab
## Run through this notebook to quickly set up an API endpoint and expose it on Flank

## 1. Get API running in this browser

---






In [ ]:
%pip install -q fastapi uvicorn pyngrok # This could take a minute

In [ ]:
import nest_asyncio
import uvicorn
import getpass
import threading
import json
import requests
from fastapi import FastAPI, Query
from typing import Optional
from pyngrok import ngrok, conf
from google.colab.output import eval_js
server_instance = None

app = FastAPI()

# Dummy sales data
sales_data = [
    {"id": 1, "date": "2023-01-01", "amount": 100.0, "customer_name": "ABC Corp"},
    {"id": 2, "date": "2023-01-02", "amount": 150.0, "customer_name": "XYZ Inc"},
    {"id": 3, "date": "2023-01-03", "amount": 200.0, "customer_name": "LMN Enterprises"},
    {"id": 4, "date": "2023-01-04", "amount": 120.0, "customer_name": "PQR Limited"},
    {"id": 5, "date": "2023-01-05", "amount": 180.0, "customer_name": "EFG Corporation"},
]

# Endpoint to get a specific sale
@app.get("/api/reports/sales")
async def generate_sales_report(
    sale_id: int = Query(..., description="Sale ID to filter the sales report")
) -> list:
    for sale in sales_data:
        if sale["id"] == sale_id:
            return [sale]
    return []

# Endpoint to get sale ids
@app.get("/api/reports/sale_ids")
async def generate_sales_report() -> list:
    return [{"id": sale["id"], "Customer Name": sale["customer_name"]} for sale in sales_data]


# Endpoint to post a new sale
@app.post("/api/sales")
async def add_sale(id: int, date: str, amount: float, customer_name: str) -> dict:
    sale = {"id": id, "date": date, "amount": amount, "customer_name": customer_name}
    sales_data.append(sale)
    return sale

# Run this API locally on port 8050
def run_server():
    global server_instance
    config = uvicorn.Config(app=app, host="0.0.0.0", port=8050)
    server_instance = uvicorn.Server(config)
    server_instance.run()

def stop_server():
    if server_instance is not None:
        server_instance.should_exit = True
        server_instance.force_exit = True
        print("Server is shutting down...")
    else:
        print("Server is not running.")

In [ ]:
local_thread = threading.Thread(target=run_server)
local_thread.start()

### What happened here??

**fastapi** is an API framework. **uvicorn** is a server that listens for API requests. We just created some simple "endpoints" around the `sales_data` that exposes that code over HTTP.

In [ ]:
# See your endpoint running locally
url = eval_js("google.colab.kernel.proxyPort(8050)")
print(f"Sales report endpoint running locally at {url}api/reports/sales?sale_id=1")

## 2. Expose that API over the public internet

We're going to use a service called ngrok to take our local API and make it available over the public internet.

**ngrok**: In a new tab, [log in or create a free ngrok account](https://dashboard.ngrok.com/get-started/your-authtoken). Once you've logged in, navigate to the [authtoken page](https://dashboard.ngrok.com/get-started/your-authtoken) if you aren't already there. Copy the Authtoken at the top of the screen, and then navigate back here to the notebook.

Next, you'll be prompted to paste in that authtoken here.

In [ ]:
print("Paste in your ngrok authtoken that you copied in the step above, then hit enter:")
conf.get_default().auth_token = getpass.getpass()

In [ ]:
ngrok_tunnel = ngrok.connect(8050)
ngrok_url = ngrok_tunnel.public_url
docs_endpoint = f"{ngrok_url}/openapi.json"
print('Endpoint exposed publicly at:', ngrok_url + "/api/reports/sales?sale_id=1")
print('Schema that Flank will read:', ngrok_url + "/openapi.json")

### Now your API is available publicly!

## 3. Get a Flank API token

[Navigate to Flank](https://www.flank.cloud/copy-token) and create an account.

A token and org id will be copied to your clipboard. As soon as you've been notified that's done, navigate back here to the notebook. Paste your token and your org id into the cell below.

In [ ]:
# Paste here!
# auth_token =
# org_id =

## 4. Get a Flank UI. Woohoo!

In [ ]:
flank_api_url = f"https://api.flank.cloud/v1/{org_id}"
headers = {"Authorization": f"Bearer {auth_token}"}
resources_payload = {
    "name": "Sales API",
    "description": "",
    "creator_org_id": org_id,
    "resource_type": "api",
    "is_public": True,
    "api": {
        "api_spec_url": docs_endpoint,
        "image_url": "",
        "api_auth_schema": ""
    },
    "db": None
}

add_resources_resp = requests.post(url=f"{flank_api_url}/resources", headers=headers, data=json.dumps(resources_payload))
add_resources_resp = add_resources_resp.json()
resource_id = add_resources_resp['resource_id']

creds_payload = {
    "org_id": org_id,
    "cred_type": "api",
    "name": "Sales API",
    "syncs": True,
    "runs": True,
    "stores": False,
    "creds_db": None,
    "creds_aws": None,
    "creds_az": None,
    "creds_api": {
        "custom_headers": "",
        "auth_type": None,
        "api_spec_url": docs_endpoint,
        "client_key": "",
        "client_secret": "",
        "bearer_token": "",
        "api_key": None,
        "username": None,
        "password": None
    },
    "resource_id": resource_id
}
creds_endpoint_resp = requests.post(url=f"{flank_api_url}/creds", headers=headers, data=json.dumps(creds_payload))
creds_endpoint_resp = creds_endpoint_resp.json()

creds_id = creds_endpoint_resp['creds_id']
sync_resource_resp = requests.get(url=f"{flank_api_url}/creds/{creds_id}/diff", headers=headers)
sync_resource_resp = sync_resource_resp.json()

kit_ids = []

for endpoint in sync_resource_resp:
  sync_endpoint_payload = {
      "cloud_id": endpoint['cloud_id'],
      "creds_id": creds_id
  }
  sync_endpoint_resp = requests.post(url=f"{flank_api_url}/syncs/cloud_id", headers=headers, data=json.dumps(sync_endpoint_payload))
  sync_endpoint_resp = sync_endpoint_resp.json()

  kit_id = sync_endpoint_resp['stage2']['kit_id']
  kit_ids.append(kit_id)

  publish_endpoint_payload = {"visibility": "store"}
  publish_endpoint_resp = requests.patch(url=f"{flank_api_url}/skinny-kits/{kit_id}", headers=headers, data=json.dumps(publish_endpoint_payload))
  publish_endpoint_resp = publish_endpoint_resp.json()

## Check it out! You've got UI for all of your endpoints.

In [ ]:
for kit_id in kit_ids:
  print(f'Check out your Flank UI at https://www.flank.cloud/store/builder?id={kit_id}')

## 5. Run and share on Flank. Woohoo!
Now, you've got a dedicated page webpage on which you can run your endpoint (we call these endpoints "commands" in Flank).

Try running your endpoint!

And just like that, you've got an interface to get your sales report.

Notice that Flank automatically found the parameters (`sale_id`). Flank will find parameters specified in API specs.

If you want to further configure the command (make the name more user-friendly, add a note to it, hide/lock certain parameters), you can do so here.

You can share your command with others and they can run your endpoint without you needing to build an interface for them.

Note that your ngrok tunnel will shut down when you close this notebook. But you can put any piece of deployed code on Flank - a stored procedure, a cloud function, a deployed API.

### What will you Flank?

In [ ]:
# If you need to shut down your local server (optional)
stop_server()